In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
#download neccessary libraries

In [ ]:
!pip install spacy==3.5.3

In [ ]:
!python -m spacy download en_core_web_sm

2024-06-20 13:02:27.193086: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-20 13:02:27.193208: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-20 13:02:27.198342: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-20 13:02:30.547858: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 22.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [ ]:
import spacy
nlp = spacy.load("en_core_web_sm")
#using spacy library to tokenaze and get the root of the words.

In [ ]:
print(nlp.Defaults.stop_words)

{'unless', 'through', 'within', 'over', 'most', 'besides', 'their', 'become', 'to', 'sixty', 'that', 'yet', 'neither', 'whose', 'becoming', 'ever', 'various', 'since', 'about', 'beside', 'give', 'us', 'put', 'above', 'seems', 'anyway', 'six', 'five', 'least', 'n’t', 'an', 'sometimes', 'mostly', 'another', '‘s', 'how', 'where', 'may', 'nothing', 'off', 'really', 'such', 'me', 'while', 'these', 'amount', 'full', 'been', 'whereafter', 'yours', 'nowhere', 'either', 'well', 'are', 'more', 'side', 'the', 'hence', 'own', 'should', 'serious', 'among', 'after', 'rather', "'ve", 'once', 'n‘t', 'somehow', 'third', 'anyone', 'somewhere', 'am', 'until', 'perhaps', 'though', 'hundred', 'regarding', 'him', 'whenever', 'why', 'upon', 'here', 'first', 'call', 'twenty', 're', "'re", 'under', 'your', 'meanwhile', 'made', 'no', 'someone', 'into', 'everywhere', 'nobody', "'m", 'anything', 'as', 'hereafter', '’m', 'not', 'those', 'which', 'never', 'than', 'due', 'latterly', 'enough', 'few', 'would', 'nine',

In [ ]:
example = nlp("apples")
lemma = example[0].lemma_
print(lemma)

apple


In [ ]:
spacy_train = pd.read_csv("/content/train.csv")


In [ ]:
spacy_train.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
spacy_train = spacy_train.fillna("")

In [ ]:
spacy_train["new_col"] = spacy_train["author"] + " " + spacy_train["title"]
#normally to get the most accurate results, we are supposed to add train_dataset["text"] to the content ingredient but this largely
#incraeses the run time up to 75 hours therefore we can simply not add train_dataset["text"] as a part of the "content" column.

In [ ]:
X_spacy = spacy_train.drop(columns="label", axis=1)
Y_spacy = spacy_train["label"]

In [ ]:
#declare a function for stemming with spacy, the parameter will be the content in string format.
#the second line, create a for loop which iterates through the words tokenized by the spacy library,if those
#words are not considered as stop words in NLP and registred in spacy default lib they are
#added to the list by token.lemma_ and evantually joined with a single space by " ".join method.
#and finally you return them.
def stemming_spacy(content):
  doc = nlp(content)
  stemmed_content = " ".join([token.lemma_ for token in doc if not token.is_stop])
  return stemmed_content

In [ ]:
spacy_train["new_col"] = spacy_train["new_col"].apply(stemming_spacy)

In [ ]:
print(spacy_train["new_col"])

0        Darrell Lucus House Dem Aide : Comey Letter Ja...
1        Daniel J. Flynn FLYNN : Hillary Clinton , Big ...
2                            Consortiumnews.com truth fire
3        Jessica Purkiss 15 Civilians kill Single Airst...
4        Howard Portnoy iranian woman jail fictional un...
                               ...                        
20795    Jerome Hudson Rapper T.I. : trump ' poster chi...
20796    Benjamin Hoffman N.F.L. Playoffs : schedule , ...
20797    Michael J. de la Merced Rachel Abrams Macy say...
20798    Alex Ansary NATO , Russia hold parallel Exerci...
20799                        David Swanson keep F-35 Alive
Name: new_col, Length: 20800, dtype: object


In [ ]:
X_spacy = spacy_train["new_col"].values
Y_spacy = spacy_train["label"].values

In [ ]:
print(X_spacy)
print(Y_spacy)

['Darrell Lucus House Dem Aide : Comey Letter Jason Chaffetz Tweeted'
 'Daniel J. Flynn FLYNN : Hillary Clinton , Big Woman Campus - Breitbart'
 'Consortiumnews.com truth fire' ...
 'Michael J. de la Merced Rachel Abrams Macy say receive Takeover Approach Hudson Bay - New York Times'
 'Alex Ansary NATO , Russia hold parallel Exercises Balkans'
 'David Swanson keep F-35 Alive']
[1 0 1 ... 0 1 1]


In [ ]:
vectorizer_spacy = TfidfVectorizer()
vectorizer_spacy.fit(X_spacy)
X_spacy = vectorizer_spacy.transform(X_spacy)

In [ ]:
print(X_spacy)

  (0, 20999)	0.416264091588138
  (0, 12172)	0.36132332190774163
  (0, 11813)	0.29326751055488953
  (0, 10716)	0.24630272270148484
  (0, 9756)	0.2200096660350064
  (0, 5447)	0.2849220774592552
  (0, 5175)	0.3576440064074609
  (0, 4238)	0.2453018368482093
  (0, 3636)	0.36535355604517383
  (0, 826)	0.31360565743455254
  (1, 22368)	0.2838041564016384
  (1, 9508)	0.19234790647233116
  (1, 7881)	0.721387824245855
  (1, 5139)	0.26865354612906167
  (1, 4039)	0.19460750691285728
  (1, 3297)	0.3829017029210215
  (1, 2885)	0.15675338044158896
  (1, 2376)	0.2842972823304079
  (2, 20906)	0.5204418982149154
  (2, 7721)	0.4979866040471177
  (2, 4494)	0.5709777074960136
  (2, 4218)	0.3938705755888242
  (3, 18677)	0.3404370026172586
  (3, 16183)	0.45630743620818
  (3, 11226)	0.24890397158923194
  :	:
  (20797, 16608)	0.27651059369909603
  (20797, 16330)	0.2509427339758817
  (20797, 13878)	0.08125180699615442
  (20797, 12993)	0.17644496511174929
  (20797, 12914)	0.29862161530915293
  (20797, 12281)	0.36

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_spacy, Y_spacy, test_size = 0.2 ,stratify=Y_spacy, random_state=42,)

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

In [ ]:
X_train_pred = model.predict(X_train)
training_acc = accuracy_score(X_train_pred, Y_train)

In [ ]:
print("accuracy: ", training_acc)

accuracy:  0.9887019230769231


In [ ]:
X_test_prediction = model.predict(X_test)
test_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print("accuracy of test : ", test_accuracy )

accuracy of test :  0.973798076923077


In [ ]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:

train_dataset = pd.read_csv('/content/train.csv')

In [ ]:
train_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
train_dataset.shape

(20800, 5)

In [ ]:
train_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [ ]:
train_dataset = train_dataset.fillna('')

In [ ]:
train_dataset["content"] = train_dataset["author"] + ' ' + train_dataset["title"]
#normally to get the most accurate results, we are supposed to add train_dataset["text"] to the content ingredient but this largely
#incraeses the run time up to 75 hours therefore we can simply not add train_dataset["text"] as a part of the "content" column.

In [ ]:
X = train_dataset.drop(columns="label",axis=1)
Y = train_dataset["label"]

In [ ]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

In [ ]:
stemmer = PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [stemmer.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  print(stemmed_content)
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
train_dataset["content"] = train_dataset["content"].apply(stemming)

Streaming output truncated to the last 5000 lines.
['richard', 'fausset', 'jonah', 'engel', 'bromwich', 'least', 'die', 'tornado', 'sweep', 'southeast', 'u', 'new', 'york', 'time']
['awr', 'hawkin', 'arm', 'citizen', 'threw', 'wallet', 'distract', 'suspect', 'lit', 'breitbart']
['ian', 'buruma', 'end', 'anglo', 'american', 'order', 'new', 'york', 'time']
['dan', 'riehl', 'keef', 'dem', 'left', 'media', 'ask', 'disavow', 'violent', 'inaugur', 'protest']
['watch', 'russia', 'launch', 'latest', 'super', 'stealth', 'submarin']
['alex', 'ansari', 'china', 'repeatedli', 'hack', 'us', 'stole', 'data', 'nuke', 'fbi', 'war', 'plan']
['red', 'ice', 'merkel', 'worri', 'islamis', 'sing', 'christma', 'carol']
['ami', 'moreno', 'break', 'hillari', 'clinton', 'maid', 'print', 'classifi', 'inform', 'truthfe']
['joel', 'b', 'pollak', 'california', 'big', 'sur', 'highway', 'close', 'massiv', 'landslid', 'breitbart']
['vindu', 'goel', 'c', 'develop', 'tool', 'spi', 'mac', 'comput', 'wikileak', 'disclosur

In [ ]:
print(train_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [ ]:
X = train_dataset["content"].values
Y = train_dataset["label"].values

In [ ]:
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [ ]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [ ]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
model = LogisticRegression()

In [ ]:
model.fit(X_train, Y_train)

LogisticRegression()

In [ ]:
X_train_prediction = model.predict(X_train)
training_accuracy = accuracy_score(X_train_prediction, Y_train)

In [ ]:
print("accuracy: ", training_accuracy)

accuracy:  0.9865985576923076


In [ ]:
X_test_prediction = model.predict(X_test)
test_accuracy = accuracy_score(X_test_prediction, Y_test)

In [ ]:
print("accuracy of test : ", test_accuracy )

accuracy of test :  0.9790865384615385


In [ ]:
X_new = X_test[11]
prediction = model.predict(X_new)
print(prediction)
if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [ ]:
print(Y_test[11])

1
